In [2]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack, hstack
import fitsio
from astropy.io import fits

In [3]:
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [55]:
ccdnamenumdict = {'S1': 25, 'S2': 26, 'S3': 27, 'S4':28,
                  'S5': 29, 'S6': 30, 'S7': 31,
                  'S8': 19, 'S9': 20, 'S10': 21, 'S11': 22, 'S12': 23,
                  'S13': 24,
                  'S14': 13, 'S15': 14, 'S16': 15, 'S17': 16, 'S18': 17,
                  'S19': 18,
                  'S20': 8, 'S21': 9, 'S22': 10, 'S23': 11, 'S24': 12,
                  'S25': 4, 'S26': 5, 'S27': 6, 'S28': 7,
                  'S29': 1, 'S30': 2, 'S31': 3,
                  'N1': 32, 'N2': 33, 'N3': 34, 'N4': 35,
                  'N5': 36, 'N6': 37, 'N7': 38,
                  'N8': 39, 'N9': 40, 'N10': 41, 'N11': 42, 'N12': 43,
                  'N13': 44,
                  'N14': 45, 'N15': 46, 'N16': 47, 'N17': 48, 'N18': 49,
                  'N19': 50,
                  'N20': 51, 'N21': 52, 'N22': 53, 'N23': 54, 'N24': 55,
                  'N25': 56, 'N26': 57, 'N27': 58, 'N28': 59,
                  'N29': 60, 'N30': 61, 'N31': 62,
                  }

fringe_dir = '/global/homes/d/djschleg/cosmo/staging/decam/DECam_CP-Fringe'
image_dir = '/global/project/projectdirs/cosmo/staging/'
surveyccd_path = '/global/project/projectdirs/cosmo/work/legacysurvey/dr9/survey-ccds-decam-dr9-cut.fits.gz'
blob_dir = '/global/project/projectdirs/desi/users/rongpu/dr9/decam_ccd_blob_mask'

sky_dir = '/global/project/projectdirs/desi/users/rongpu/dr9/fringe/smooth_sky'
plot_dir = '/global/project/projectdirs/desi/users/rongpu/dr9/fringe/plots'
output_dir = '/global/project/projectdirs/desi/users/rongpu/dr9/fringe/data'

# Load CCD list
ccd_columns = ['image_filename', 'image_hdu', 'expnum', 'ccdname', 'filter', 'mjd_obs', 'ra', 'dec', 'skyrms', 'ccdraoff', 'ccddecoff', 'ccd_cuts']
ccd = fitsio.read(surveyccd_path, columns=ccd_columns)
# ccd = fitsio.read(surveyccd_path)
ccd = Table(ccd)
mask = ccd['ccd_cuts']==0
mask &= ccd['filter']=='z' # include only z-band images
ccd = ccd[mask]
print(len(ccd))

# Find CCDs around some MJD
mask = (ccd['mjd_obs']>(57815-4)) & (ccd['mjd_obs']<(57815+4)) # DECaLS observing run starting Feb 28, 2017
mask |= ((ccd['mjd_obs']>(58359-2)) & (ccd['mjd_obs']<(58359+27))) # Starting Aug 28, 2018
mask |= ((ccd['mjd_obs']>(58423-2)) & (ccd['mjd_obs']<(58423+30))) # Two runs starting Oct 28, 2018
mask |= ((ccd['mjd_obs']>(57893-2)) & (ccd['mjd_obs']<(57893+30))) # Two runs starting May 18, 2017
ccd = ccd[mask]
print(len(ccd))

1148265
164459


In [56]:
# Find the CCDs whose blobmask files exist
ccd_mask = np.zeros(len(ccd), dtype=bool)
for ccd_index in range(len(ccd)):
    str_loc = str.find(ccd['image_filename'][ccd_index].strip(), '.fits')
    img_filename_base = ccd['image_filename'][ccd_index].strip()[:str_loc]
    blob_path = os.path.join(blob_dir, 'blob_mask', img_filename_base+'-blobmask.npz')
    if os.path.isfile(blob_path):
        ccd_mask[ccd_index] = True
print(np.sum(ccd_mask)/len(ccd_mask))
ccd = ccd[ccd_mask]
print(len(ccd))

0.831605445734
136765


In [57]:
t = Table()
t['expnum'], t['counts'] = np.unique(ccd['expnum'], return_counts=True)

# Only keep exposures with all 61 CCDs
mask = t['counts']==61
mask_remove = ~np.in1d(ccd['expnum'], t['expnum'][mask])
print(np.sum(mask_remove), np.sum(~mask_remove))
ccd = ccd[~mask_remove]

5127 131638


In [58]:
print(len(np.unique(ccd['ccdname'])))
print(' '.join(np.unique(ccd['ccdname'])))

61
N1  N10 N11 N12 N13 N14 N15 N16 N17 N18 N19 N2  N20 N21 N22 N23 N24 N25 N26 N27 N28 N29 N3  N31 N4  N5  N6  N7  N8  N9  S1  S10 S11 S12 S13 S14 S15 S16 S17 S18 S19 S2  S20 S21 S22 S23 S24 S25 S26 S27 S28 S29 S3  S30 S31 S4  S5  S6  S7  S8  S9 


In [59]:
# For simplicity choose one CCD
mask = ccd['ccdname']=='N10'
ccd = ccd[mask]
print(len(ccd))

# Identify the observing date of each CCD
str_loc = np.char.find(np.array(ccd['image_filename'], dtype='str'), '/CP201')
ccd['obs_date'] = np.array([ccd['image_filename'][i][str_loc[i]+1:str_loc[i]+11] for i in range(len(ccd))])
t = Table()
t['obs_date'], t['counts'] = np.unique(ccd['obs_date'], return_counts=True)

# Require a minimum number of CCDs (since scipy gaussian_filter does not handle NAN)
mask = t['counts']<50
mask_remove = np.in1d(ccd['obs_date'], t['obs_date'][mask])
print(np.sum(mask_remove), np.sum(~mask_remove))
ccd = ccd[~mask_remove]

# Also remove nights with too many exposures
mask = t['counts']>=120
mask_remove = np.in1d(ccd['obs_date'], t['obs_date'][mask])
print(np.sum(mask_remove), np.sum(~mask_remove))
ccd = ccd[~mask_remove]

2158
340 1818
413 1405


In [60]:
t = Table()
t['obs_date'], t['n_exposure'] = np.unique(ccd['obs_date'], return_counts=True)
t

obs_date,n_exposure
str10,int64
CP20170302,60
CP20170303,59
CP20170304,82
CP20170305,114
CP20170519,64
CP20170520,86
CP20180828,111
CP20180829,67
CP20180831,117


In [61]:
np.array(t['obs_date'])

array(['CP20170302', 'CP20170303', 'CP20170304', 'CP20170305',
       'CP20170519', 'CP20170520', 'CP20180828', 'CP20180829',
       'CP20180831', 'CP20180903', 'CP20180908', 'CP20180913',
       'CP20180914', 'CP20180916', 'CP20180917', 'CP20180920', 'CP20180921'],
      dtype='<U10')

In [62]:
obs_date_list = ['CP20170302', 'CP20170303', 'CP20170304', 'CP20170305', 'CP20170519', 'CP20170520', 'CP20180828', 'CP20180829', 'CP20180831', 'CP20180903', 'CP20180908', 'CP20180913', 'CP20180914', 'CP20180916', 'CP20180917', 'CP20180920', 'CP20180921']
np.in1d(t['obs_date'], obs_date_list)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)